# Code search with Qdrant

This is a notebook demonstrating how to implement a code search mechanism using two different neural encoders - one general purpuse, and another trained specifically for code. Let's start with installing all the required dependencies.

In [ ]:
!pip install qdrant-client inflection sentence-transformers optimum onnx

We are going to work with [Qdrant source code](https://github.com/qdrant/qdrant) that has been already converted into chunks. If you want to do it for a different project, please consider using one of the [LSP implementations](https://microsoft.github.io/language-server-protocol/) for your programming language. It should be fairly easy to build similar structures with the help of these tools.

In [ ]:
!wget https://storage.googleapis.com/tutorial-attachments/code-search/structures.jsonl

In [ ]:
import json

structures = []
with open("structures.jsonl", "r") as fp:
    for i, row in enumerate(fp):
        entry = json.loads(row)
        structures.append(entry)

structures[0]

We will use two different neural encoders - `all-MiniLM-L6-v2` and `jina-embeddings-v2-base-code`. Since the first one is trained for general purposes, and more natural language, there is a need to convert code into more human-friendly text representation. This normalization gets rid of language specifics, so the output looks more like a description of the particular code structure.

In [ ]:
import inflection
import re

from typing import Dict, Any

def textify(chunk: Dict[str, Any]) -> str:
    """
    Convert the code structure into natural language like representation.

    Args:
        chunk (dict): Dictionary-like representation of the code structure
            Example: {
                "name":"await_ready_for_timeout",
                "signature":"fn await_ready_for_timeout (& self , timeout : Duration) -> bool",
                "code_type":"Function",
                "docstring":"= \" Return `true` if ready, `false` if timed out.\"",
                "line":44,
                "line_from":43,
                "line_to":51,
                "context":{
                    "module":"common",
                    "file_path":"lib/collection/src/common/is_ready.rs",
                    "file_name":"is_ready.rs",
                    "struct_name":"IsReady",
                    "snippet":"    /// Return `true` if ready, `false` if timed out.\n    pub fn await_ready_for_timeout(&self, timeout: Duration) -> bool {\n        let mut is_ready = self.value.lock();\n        if !*is_ready {\n            !self.condvar.wait_for(&mut is_ready, timeout).timed_out()\n        } else {\n            true\n        }\n    }\n"
                }
            }

    Returns:
        str: A simplified natural language like description of the structure with some context info
            Example: "Function Await ready for timeout that does Return true if ready false if timed out defined as Fn await ready for timeout self timeout duration bool defined in struct Isready module common file is_ready rs"
    """
    # Get rid of all the camel case / snake case
    # - inflection.underscore changes the camel case to snake case
    # - inflection.humanize converts the snake case to human readable form
    name = inflection.humanize(inflection.underscore(chunk["name"]))
    signature = inflection.humanize(inflection.underscore(chunk["signature"]))

    # Check if docstring is provided
    docstring = ""
    if chunk["docstring"]:
        docstring = f"that does {chunk['docstring']} "

    # Extract the location of that snippet of code
    context = (
        f"module {chunk['context']['module']} "
        f"file {chunk['context']['file_name']}"
    )
    if chunk["context"]["struct_name"]:
        struct_name = inflection.humanize(
            inflection.underscore(chunk["context"]["struct_name"])
        )
        context = f"defined in struct {struct_name} {context}"

    # Combine all the bits and pieces together
    text_representation = (
        f"{chunk['code_type']} {name} "
        f"{docstring}"
        f"defined as {signature} "
        f"{context}"
    )

    # Remove any special characters and concatenate the tokens
    tokens = re.split(r"\W", text_representation)
    tokens = filter(lambda x: x, tokens)
    return " ".join(tokens)

Here is how the same structure looks like, after performing the normalization step:

In [ ]:
textify(structures[0])

Let's do it for all the structures at once:

In [ ]:
text_representations = list(map(textify, structures))

Created text representations might be directly used as an input to the `all-MiniLM-L6-v2` model.

In [ ]:
from sentence_transformers import SentenceTransformer

nlp_model = SentenceTransformer("all-MiniLM-L6-v2")
nlp_embeddings = nlp_model.encode(
    text_representations, show_progress_bar=True,
)
nlp_embeddings.shape

As a next step, we are going to extract all the code snippets to a separate list. This will be an input to the different model we want to use.

In [ ]:
code_snippets = [
    structure["context"]["snippet"]
    for structure in structures
]
code_snippets[0]

The `jina-embeddings-v2-base-code` model is available for free, but requires accepting the rules on [the model page](https://huggingface.co/jinaai/jina-embeddings-v2-base-code). Please do it first, and put the key below.

In [ ]:
# You have to accept the conditions in order to be able to access Jina embedding
# model. Please visit https://huggingface.co/jinaai/jina-embeddings-v2-base-code
# to accept the rules and generate the access token in your account settings:
# https://huggingface.co/settings/tokens

HF_TOKEN = "THIS_IS_YOUR_TOKEN"

Once the token is ready, we can pass the code snippets through the second model.

In [ ]:
code_model = SentenceTransformer(
    "jinaai/jina-embeddings-v2-base-code",
    token=HF_TOKEN,
    trust_remote_code=True
)
code_model.max_seq_length = 8192  # increase the context length window
code_embeddings = code_model.encode(
    code_snippets, batch_size=4, show_progress_bar=True,
)
code_embeddings.shape

Created embeddings have to be indexed in a Qdrant collection. For that, we need a running instance. The easiest way is to deploy it using the [Qdrant Cloud](https://cloud.qdrant.io/). There is a free tier 1GB cluster available, but you can alternatively use [a local Docker container](https://qdrant.tech/documentation/quick-start/), but running it in Google Colab might require installing Docker first.

In [ ]:
QDRANT_URL = "https://my-cluster.cloud.qdrant.io:6333" # http://localhost:6333 for local instance
QDRANT_API_KEY = "THIS_IS_YOUR_API_KEY" # None for local instance

In [ ]:
from qdrant_client import QdrantClient, models

client = QdrantClient(QDRANT_URL, api_key=QDRANT_API_KEY)
client.create_collection(
    "qdrant-sources",
    vectors_config={
        "text": models.VectorParams(
            size=nlp_embeddings.shape[1],
            distance=models.Distance.COSINE,
        ),
        "code": models.VectorParams(
            size=code_embeddings.shape[1],
            distance=models.Distance.COSINE,
        ),
    }
)

Our collection should be created already. Let's finally index all the data.

In [ ]:
import uuid

points = [
    models.PointStruct(
        id=uuid.uuid4().hex,
        vector={
            "text": text_embedding,
            "code": code_embedding,
        },
        payload=structure
    )
    for text_embedding, code_embedding, structure in zip(nlp_embeddings, code_embeddings, structures)
]
len(points)

In [ ]:
client.upload_points(
    "qdrant-sources",
    points=points,
    batch_size=64,
)

If you want to check if all the points were sent, counting them might be the easiest idea.

In [ ]:
client.count("qdrant-sources")

If you, however, want to know how the count endpoint works internally in the Qdrant server, that might be a question to ask.

In [ ]:
query = "How do I count points in a collection?"

First of all, let's use one model at a time. Let's start with the general purpose one. We could probably omit the call to the `textify` function, but in general we should pass the input through the same process like we did for the documents.

In [ ]:
hits = client.search(
    "qdrant-sources",
    query_vector=(
        "text", nlp_model.encode(textify(query)).tolist()
    ),
    limit=5,
)
for hit in hits:
    print(
        "| ",
        hit.payload["context"]["module"], " | ",
        hit.payload["context"]["file_name"], " | ",
        hit.score, " | `",
        hit.payload["signature"], "` |"
    )

The results obtained with the code specific model should be different.

In [ ]:
hits = client.search(
    "qdrant-sources",
    query_vector=(
        "code", code_model.encode(query).tolist()
    ),
    limit=5,
)
for hit in hits:
    print(
        "| ",
        hit.payload["context"]["module"], " | ",
        hit.payload["context"]["file_name"], " | ",
        hit.score, " | `",
        hit.payload["signature"], "` |"
    )

In reality, we implemented the system with two different models, as we want to combine the results coming from both of them. We can do it with a batch request, so there is just a single call to Qdrant.

In [ ]:
results = client.search_batch(
    "qdrant-sources",
    requests=[
        models.SearchRequest(
            vector=models.NamedVector(
                name="text",
                vector=nlp_model.encode(textify(query)).tolist()
            ),
            with_payload=True,
            limit=5,
        ),
        models.SearchRequest(
            vector=models.NamedVector(
                name="code",
                vector=code_model.encode(query).tolist()
            ),
            with_payload=True,
            limit=5,
        ),
    ]
)
for hits in results:
    for hit in hits:
        print(
            "| ",
            hit.payload["context"]["module"], " | ",
            hit.payload["context"]["file_name"], " | ",
            hit.score, " | `",
            hit.payload["signature"], "` |"
        )

Last but not least, if we want to improve the diversity of the results, grouping them by the module might be a good idea.

In [ ]:
results = client.search_groups(
    "qdrant-sources",
    query_vector=(
        "code", code_model.encode(query).tolist()
    ),
    group_by="context.module",
    limit=5,
    group_size=1,
)
for group in results.groups:
    for hit in group.hits:
        print(
            "| ",
            hit.payload["context"]["module"], " | ",
            hit.payload["context"]["file_name"], " | ",
            hit.score, " | `",
            hit.payload["signature"], "` |"
        )

For a more detailed guide, please check our [code search tutorial](https://qdrant.tech/documentation/tutorials/code-search/) and [code search demo](https://github.com/qdrant/demo-code-search).